In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from freedom.utils.dataset import Data
from freedom.utils.callback import Save

from freedom.neural_nets.chargenet import get_chargenet

In [2]:
data = Data(dirs=['/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols','/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols'])

In [3]:
chargenet = get_chargenet(data.labels)

In [4]:
chargenet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
chargenet_trafo (chargenet_traf (None, 9)            0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           320         chargenet_trafo[0][0]            
______________________________________________________________________________________________

In [5]:
optimizer = tf.keras.optimizers.Adam(0.0001)
chargenet.compile(loss='binary_crossentropy', optimizer=optimizer)

In [6]:
train_id = 'ChargeNet_logE_' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [7]:
callbacks = []
callbacks.append(Save(save_every=20, path_template='../../freedom/resources/models/'+train_id+'/epoch_%i', ))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='../../freedom/resources/logs/'+train_id, histogram_freq=1))

In [8]:
train_data, test_data = data.get_chargenet_data(train_batch_size=4096,
                                                test_batch_size=4096,
                                                )

In [9]:
hist = chargenet.fit(train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                     initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000


KeyboardInterrupt: 

In [ ]:
# best valid loss L7 0.53....52
# L3: 0.48 ish
# L3 bigger net: 0.48...0.47 with 12+14,
# after epoch 400 reduced lr to 0.00001
# 0.45 with 256 and 12+14, batch size 4096